<a href="https://colab.research.google.com/github/AlcoholWolf/py-play/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch09. 전처리 실습 - Titanic
---
* 날짜:
* 이름:

**소개**

  - [타이타닉 데이터셋](https://www.kaggle.com/c/titanic) 전처리를 진행 합니다.
  - [파이썬 데이터전처리 실습-Titanic](https://wikidocs.net/86297)를 참고하여 전처리를 진행 합니다.
  - 향후 이 코드를 이어서 사용하니 코드를 잘 정리해 주세요.


기준 : %는 곱계산입니다.
생존 여부가 1 인 경우 +100
티켓 등급이 1 인 경우 +25
티켓 등급이 2 인 경우 +35
티켓 등급이 3 인 경우 +25
성별이 남자인 경우    +50
성별이 여자인 경우    +50

나이가 20~35 사이인 경우            +50
나이가 12~20/35~44 사이인 경우      +25
나이가 0~12/44~100 사이인 경우      +10
배우자,형제가 없는 경우             +50
부모,자식이 없는 경우               +50
만약 부모, 형제, 자식, 배우자가 있다면 -25 * N

선실이 구명보트, 선체의 갑판과 가까운 경우                 +75
출구와 선실(자신)이 다른 선실(B) 몇개를 지나야 있는 경우   -20 * B
승선한 항구가 침몰 지점에서 가장 가깝거나 먼 경우          +100
(멀다는건 그만큼 체력을 비축햇을것이다는것. 단 출항 시간-침몰시간이 3일 이내라면 가까운 경우로 변경.)
승선 요금이 기준(1등급의 평균)의 2배가 넘는 경우

## 데이터 전처리
---

In [93]:
# 조건 1 : 주석이 2줄 이상 있을 때  =  해당 첫 줄은 리뷰, 설명이며,    2번~ 줄은 상세 내용 입니다.
#
# 조건 2 : 주석이 1줄 있을 때  =  해당 첫 줄은 상세 내용 입니다.    더하여 2줄 이상 있더라도 전체가 상세 내용일 경우도 있습니다.
#
# 조건 3 : 만약 주석이 2줄 이상 있지만, 앞부분이 정말 많이 띄워져 있을 경우는 바로 위 내용과 연결되는 부분입니다. 
#          if: 하고 띄우는 방식과 비슷합니다.
#
# 조건 4 : 만약 명령 값(실제 코딩줄) 뒤에 주석과 번호가 달려있다면, 
#          그리고 위에 쓰여져 있는 주석 앞에도 번호가 있다면 명령 값과 해당 주석이 연결(해당 주석을 설명) 하는 주석 이라고 봐주시면 됩니다.
#
# 조건 5 : 만약 명령 값(실제 코딩줄) 뒤에 주석이 달려있다면, 코딩중 해당 명령 값에 대한 짧은 주석을 다는 것이라고 봐주시면 됩니다.
# 
# 주석 1 : 제가 조건을 설정하고 정리를 하는 편이지만 시간이 부족하므로 2차 검토까지만 하고 제출하게 될 것 같으므로, 
#          오류 또는 탈자(주석에 대한) 는 '이곳에 오탈자가 있엇다'라고 말해주시면 추후 정리할때 편합니다.

In [83]:
# survival - 생존여부 ( 0 = No, 1 = Yes )
# pclass - 티켓 등급 ( 1 = 1st, 2 = 2nd, 3 = 3rd )
# sex - 성별
# Age - 나이
# sibsp - 타이타닉에 승선한 형제자매 또는 배우자의 수
# parch - 타이타닉에 승선한 부모 - 자식의 수
# ticket - 티켓 번호
# fare - 승선 요금
# cabin - 선실 번호
# embarked - 승선 항구 이름 ( C = Cherbourg, Q = Queenstown, S = Southampton )

In [ ]:
# 기본 지정
import pandas as pd
import numpy as np
# .csv 파일 형식 불러오기
train = pd.read_csv('/content/train.csv')

# 곁측치, 빈 값을 호출, 0이라면 없음 상수면 있음.
# isnull은 비어있는 값, 즉 null인 슬롯 하나당 1 반환, sum으로 각 개체 열합
nulltrain = train.isnull().sum()

# 최상단 5줄(분류자 포함) 로드
# head는 맨 위(분류자 또한 1번 줄)부터 5번 줄까지 반환
printtrain = train.head(5)

# 최소 최대 평균 등의 분포도 로드
# describe는 각 열의 측정값(최소 최대 총합 평균 등등)을 반환
destrain = train.describe()

# .dtypes 메소드를 사용하여 해당 열들의 객체 타입 로드
# dtypes은 기준이 되는 열이나 행의 객체 타입 반환
dtypetrain = train.dtypes

# 객실의 칸 정보(단일값) 과, 승선한 항구의 정보(3종류 + 눌)
# unique는 해당 열(기준값)의 겹치지 않은 데이터값(단일값)을 반환
cabintrain = train.Cabin.unique()
embarkedtrain = train.Embarked.unique()

# 생존 인원, 비생존 인원, 총 인원
# 생존은 1, 비생존은 0으로 저장되어있으므로, 생존 항목 모든 값을 더하면 생존 인원수 반환,
#           총 행의 갯수 에서 생존 인원 수를 뺴면 비생존 인원 수 반환.
survival = train.Survived.sum()
n_survival = train.shape[0] - survival
print(f'총 인원 {survival+n_survival}')
print(f'생존자 {survival}\n사망자 {n_survival}')

# 1,2,3등급의 인원(신분) 분류
# Pclass 열 중 1,2,3 의 value값에 해당하는 인원 을 counts로 세어서 반환 
pclasstrain = train['Pclass'].value_counts()
print(f'1등급 인원 {pclasstrain[1]}\n2등급 인원 {pclasstrain[2]}\n3등급 인원 {pclasstrain[3]}')

# 남성/여성으로 분류
# Sex의 male값과 female값을 counts해서 반환
sextrain = train['Sex'].value_counts()
print(f'남성 {sextrain[1]}\n여성 {sextrain[0]}')

# 2개의 곁측치는 Southampton로 변환
# 비어있는 null값에 S 채워넣고(fillna), S, C, Q 값을 counts해서 반환
train['Embarked'] = train['Embarked'].fillna('S')
embartrain = train['Embarked'].value_counts()
print(f'Southampton {embartrain[0]}\nCherbourg {embartrain[1]}\nQueenstown {embartrain[2]}')

# median으로 Age의 중간값을 로드해서 fillna함수를 사용하여 곁측치(null)을 Age의 중간값으로 변환
train['Age'] = train['Age'].fillna(train['Age'].median())

# name에서 신분을 나타내는 단어를 title에 할당
# extract메소드를 사용하여 정규 식을 넣으면 해당하는 문자열을 반환
# replace메소드를 이용, 기준 리스트의 문자열을 2번 인자 문자열로 치환
train['Title'] = train['Name'].str.extract('([A-Za-z]+)\.', expand=False)
train['Title'] = train['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
train['Title'] = train['Title'].replace(['Jonkheer', 'Master'], 'Master')
train['Title'] = train['Title'].replace(['Don', 'Sir', 'the Countess', 'Lady', 'Dona'], 'Royalty')
train['Title'] = train['Title'].replace(['Mme', 'Ms', 'Mrs'], 'Mrs')
train['Title'] = train['Title'].replace(['Mlle', 'Miss'], 'Miss')
train['Title'] = train['Title'].replace(['Mr'], 'Mr')
train['Title']


# 나이형 변수 선언
# 0~18,19~25,26~35,36~60,61~100 순으로 'Baby', 'Youth', 'YoungAdult', 'MiddleAged', 'Senior' 그룹에 각각 할당.
# 할당한 그룹마다의 객체 수 카운트
bin = [0, 18, 25, 35, 60, 100]
group_names = ['Baby', 'Youth', 'YoungAdult', 'MiddleAged', 'Senior']
train['AgeGroup'] = pd.cut(train['Age'], bins=bin, labels=group_names)
agetrain = train['AgeGroup'].value_counts()
print(agetrain)


# 변수 지정 슬롯(순서가 다른곳이어도 이곳에 변수 모으기)
# 1 | 변수y에 학습할 목표변수인 Survived필드값을 담기.
y = train.Survived

In [92]:
# // 종합 정리 //

# 총 인원       891

# 생존률
# 생존자        342
# 사망자        549
# 생존 퍼센트율  38

# 신분
# 1등급 인원    216
# 2등급 인원    184
# 3등급 인원    491

# 성별
# 남성          314
# 여성          577

# 승선한 항구
# Southampton   646
# Cherbourg     168
# Queenstown     77

# 나이
# YoungAdult    373
# MiddleAged    195
# Youth         162
# Baby          139
# Senior         22
# 평균 나이      30